## 1.加载keras模块

In [15]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D,ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input, VGG16
from keras import backend as K
import numpy as np

### 加载预训练权值



### 定义新加入的layers

In [14]:
img_width, img_height = 150, 150
if K.image_data_format() == 'channels_first':#返回默认的图像的维度顺序
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
# 定义网络输入层shape
input_tensor = Input(shape=input_shape)
vgg16_weight=r'H:\AI_kejian\11.7\zuoye\vgg16_weights.h5'
#声明VGG网络，并指定加载imagenet预训练模型
model = VGG16(weights=None, #模型初始化的权重检查点
                           include_top=False,#不包含全连接层
                           input_tensor=input_tensor)
#向VGG网络追加一些全连接层和输出层
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
#指定输出层为2个神经元做2分类，激活函数选用softmax
top_model.add(Dense(2, activation='softmax'))

#把上述网络每一层layer拷贝至一个新的网络，形成一个整体网络
new_model = Sequential()
for l in model.layers:
    new_model.add(l)
new_model.add(top_model)

### 设置不需微调的layers的trainable属性
并调用compile函数重新编译网络

In [ ]:
from keras.optimizers import SGD
#在新的整体网络上设置对所有原VGG层均不参与微调
for layer in new_model.layers[:]:
    layer.trainable = False  #冻结卷积基
new_model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=1e-4, momentum=0.9),
              #优化器，随机梯度下降（学习率，动量参数）
              metrics=['accuracy'])
              #列表，包含评估模型在训练和测试时的性能的指标
#compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
#注意此处应选用何种loss function

### 定义ImageDataGenerator


In [ ]:
train_data_dir = r'H:\AI_kejian\11.06\dogs-vs-cats\train'
validation_data_dir = r'H:\AI_kejian\11.06\dogs-vs-cats\validation'
nb_train_samples = 10835 #用于训练的样本数
nb_validation_samples = 4000
epochs = 1
batch_size = 20  #要设置成能被上面两个数整除的数字


# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,#缩放
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)#布尔值，进行随机水平翻转

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)
#此处调用flow_from_directory函数生成train_generator和validation_generator,注意设置class_mode
train_generator = train_datagen.flow_from_directory(
    train_data_dir,#目标目录
    target_size=(img_width, img_height),#所有图像调整为150*150
    batch_size=batch_size,
    class_mode='categorical')
    #决定返回的标签数组的形式，这里是返回2D的one-hot编码标签

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')



### 训练模型



In [ ]:
#利用python的生成器，逐个生成数据的batch并进行训练
new_model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    #当生成器返回这个参数次数据时计一个epoch结束，执行下一个epoch
    epochs=epochs,#数据迭代的轮数
    validation_data=validation_generator,#生成验证集的生成器
    validation_steps=nb_validation_samples // batch_size)
    #当前一个参数为生成器时，本参数指定验证集的生成器返回次数

### 使用训练后模型预测图像





In [16]:
import cv2
img = cv2.resize(cv2.imread(r'H:\AI_kejian\11.06\dogs-vs-cats\test\1.jpg'), (img_width, img_height)).astype(np.float32)
#使用imread读取rgb图片时，得到的array的shape是（高，宽，（B，G，R））
# img[:,:,0] -= 103.939
# img[:,:,1] -= 116.779
# img[:,:,2] -= 123.68
#img = img.transpose((2,0,1))
x = img_to_array(img)

x = np.expand_dims(x, axis=0)#扩展数组的形状，在x数组

#x = preprocess_input(x)

score = new_model.predict(x)


print(score)

[[0. 1.]]


###此处尝试只锁定VGG头四层

In [ ]:
#在新的整体网络上设置原VGG前四层不参与微调
from keras.optimizers import SGD
for layer in new_model.layers[:4]:
    layer.trainable = False

new_model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

#编译网络后进行训练
train_data_dir = r'H:\AI_kejian\11.06\dogs-vs-cats\train'
validation_data_dir = r'H:\AI_kejian\11.06\dogs-vs-cats\validation'
nb_train_samples = 10835 #用于训练的样本数
nb_validation_samples = 4000
epochs = 1
batch_size = 20  #要设置成能被上面两个数整除的数字


# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,#缩放
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)#布尔值，进行随机水平翻转

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)
#此处调用flow_from_directory函数生成train_generator和validation_generator,注意设置class_mode
train_generator = train_datagen.flow_from_directory(
    train_data_dir,#目标目录
    target_size=(img_width, img_height),#所有图像调整为150*150
    batch_size=batch_size,
    class_mode='categorical')
    #决定返回的标签数组的形式，这里是返回2D的one-hot编码标签

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')
new_model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    #当生成器返回这个参数次数据时计一个epoch结束，执行下一个epoch
    epochs=epochs,#数据迭代的轮数
    validation_data=validation_generator,#生成验证集的生成器
    validation_steps=nb_validation_samples // batch_size)
    #当前一个参数为生成器时，本参数指定验证集的生成器返回次数

In [ ]:
#使用训练后模型预测图像
import cv2
img = cv2.resize(cv2.imread(r'H:\AI_kejian\11.06\dogs-vs-cats\test\2.jpg'), (img_width, img_height)).astype(np.float32)
#使用imread读取rgb图片时，得到的array的shape是（高，宽，（B，G，R））
# img[:,:,0] -= 103.939
# img[:,:,1] -= 116.779
# img[:,:,2] -= 123.68
#img = img.transpose((2,0,1))
x = img_to_array(img)

x = np.expand_dims(x, axis=0)#扩展数组的形状，在x数组

#x = preprocess_input(x)

score = new_model.predict(x)


print(score)